In [5]:
%matplotlib qt5
%load_ext autoreload
%autoreload 2
from __future__ import print_function

import os, sys, fnmatch, math
import cv2, time
import numpy as np
import argparse, pprint
from matplotlib import pyplot as plt

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, IntSlider, Layout, interact_manual,interact, HBox, Layout,VBox

pp = pprint.PrettyPrinter(indent=4)
homeDir = os.path.abspath('')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
k = 0.1  # look forward gain
Lfc = 1.0  # look-ahead distance
Kp = 1.0  # speed propotional gain
dt = 0.1  # [s]
b = 2.9  # [m] wheel base of vehicle
epsilon = 0.5
angles = []
show_animation = True

class State:
    def __init__(self, x=0.0, y=0.0, yaw=0.0, v=0.0):
        self.x = x
        self.y = y
        self.yaw = yaw
        self.v = v


def update(state, a, delta):
    state.x = state.x + state.v * math.cos(state.yaw) * dt
    state.y = state.y + state.v * math.sin(state.yaw) * dt
    state.yaw = state.yaw + state.v / b * math.tan(delta) * dt
    state.v = state.v + a * dt
    return state

def calcAvoidance(state, obs, dc=2.0, d0=1):
    xk = state.x;    yk = state.y;    yawk = state.yaw
    dxs = obs[:,0] - xk;        dys = obs[:,1] - yk
    ds = np.sqrt(dxs ** 2 + dys ** 2)
    
    fx, fy = 0,0
    d = 0
    angD = -180.0
    flag_skip = False
    Ravoids = []
    for i,dist in enumerate(ds):
        if(dist<=dc):
            d = dist
            dx = obs[i,0] - xk
            dy = obs[i,1] - yk
            alpha_i = math.atan2(dy,dx) - yawk
            angD = np.rad2deg(alpha_i)
#             print("Angle to Obstacle: %.3f" % angD)
            angles.append(angD)
            if(angD > 90.0 or angD < -90.0):
                flag_skip = True
            holo_dist_i = (dist*alpha_i)/math.sin(alpha_i)
            c1 = dc+d0
            c2 = holo_dist_i + d0
            const1 = -((1/(c1*c1))-(1/(c2*c2)))
            fxi = const1*(dx/dist)
            fyi = const1*(dy/dist)
            Fi = np.sqrt(fxi*fxi+fyi*fyi)
            fx += fxi;        fy += fyi
            ka = (2/(Fi*Lfc))+5
            if(alpha_i == 0): Ravoids.append(1/(ka*Fi))
            else: Ravoids.append((-1*np.sign(alpha_i))/(ka*Fi))
   
    if(len(Ravoids) is not 0):
        R = np.max(Ravoids)
        Kavoid = 1/(R)
        gain = (d/dc)
        gain = 1 - gain
        Kavoid = Kavoid * (gain)
    else:
        Kavoid = 0
    if(flag_skip):
        Kavoid = 0
    return Kavoid

def PIDControl(target, current):
    return Kp * (target - current)

def update_target(state, targets, pind, obs):
    nMax = targets.shape[0]

    tx = targets[pind,0];    ty = targets[pind,1]
    dx = tx - state.x;       dy = ty - state.y
    dist = np.sqrt(dx*dx + dy*dy)
    if(dist <= epsilon): ind = pind + 1
    else: ind = pind
    if ind >= nMax: ind = nMax - 1

    alpha = math.atan2(ty - state.y, tx - state.x) - state.yaw
    if state.v < 0: alpha = math.pi - alpha

    Lf = k * state.v + Lfc
    Ktrack = (2*math.sin(alpha))/Lf
    Kavoid = calcAvoidance(state,obs,5)
    Ktotal = Ktrack + Kavoid
#     print(Ktotal)
    stre = ("\rKtrack, Kavoid: %.3f, %.3f"% (Ktrack,Kavoid))
    print(stre,end="")
    sys.stdout.flush()
#     delta = math.atan2(2.0 * b * math.sin(alpha) / Lf, 1.0)
    delta = math.atan2(Ktotal * b, 1.0)
    return delta, ind

history = []
def main():
    cx = np.arange(0, 50, 10)
    cy = [math.sin(ix / 5.0) * ix / 2.0 for ix in cx]
    targets = np.array([cx, cy]).T
      
    ox = [5,15]
    oy = [2.5,-1.3]
    obs = np.array([ox,oy]).T
    target_speed = 10.0 / 3.6  # [m/s]
    T = 100.0  # max simulation time

    # initial state
    state = State(x=-0.0, y=-3.0, yaw=0.0, v=0.0)

    lastIndex = len(cx) - 1
    time = 0.0
    x = [state.x];    y = [state.y];    yaw = [state.yaw];    v = [state.v];    t = [0.0]
    target_ind = 0

    history.append([t,x,y,yaw, v])
    while T >= time and lastIndex > target_ind:
        ai = PIDControl(target_speed, state.v)
        di, target_ind = update_target(state, targets, target_ind,obs)
        state = update(state, ai, di)

        time = time + dt

        x.append(state.x)
        y.append(state.y)
        yaw.append(state.yaw)
        v.append(state.v)
        t.append(time)
        history.append([time,state.x,state.y,state.yaw, state.v])
        
        if show_animation:  # pragma: no cover
            plt.cla()
            plt.plot(cx, cy, ".r", label="course")
            plt.plot(ox, oy, "s", label="obstacle")
            plt.plot(x, y, "-b", label="trajectory")
            plt.plot(cx[target_ind], cy[target_ind], "xg", label="target")
            plt.axis("equal")
            plt.grid(True)
            plt.title("Speed[km/h]:" + str(state.v * 3.6)[:4])
            plt.pause(0.001)

    # Test
    assert lastIndex >= target_ind, "Cannot goal"

    if show_animation:  # pragma: no cover
        plt.cla()
        plt.plot(cx, cy, ".r", label="course")
        plt.plot(ox, oy, "s", label="obstacle")
        plt.plot(x, y, "-b", label="trajectory")
        plt.legend()
        plt.xlabel("x[m]")
        plt.ylabel("y[m]")
        plt.axis("equal")
        plt.grid(True)

In [ ]:
print("Pure pursuit path tracking simulation start")
main()

In [ ]:
for i in range(29):
    stre = ("\rAngle: %d"%i)
    print(stre,end="")
    time.sleep(0.5)
    sys.stdout.flush()

In [12]:
np.rad2deg(0.9326692496425261)

53.438011686150105

In [13]:
(2*math.sin(np.deg2rad(90)))/0.5

4.0